# Metagenomics

## Intro

Vervuiling is een groot probleem in de moderne samenleving en een van de grootste bijdrager daarbij is plastic afval. Gelukkig is daar een alternatief voor gevonden: Bioplastic. Bioplastics zouden beter plastic zijn dan plastic, omdat in tegenstelling tot plastic bioplastics zijn gemaakt van biologisch afbreekbare materialen. PHA is hier een goed voorbeeld van, die breekt binnen 90 dagen helemaal af in basis grond. Maar dat betekend niet dat bioplastic perfect is, het kan nog steeds schadelijk zijn voor de grond waar het in beland. Daarom gaan wij hier onderzoeken wat voor effect PHA heeft op micro-organismen in de grond.


## Data

We hebben minION data met drie soorten sample: controle, met bioplastic pallets en met bioplastic poeder.

## Tool analyse

*8-3-2025*
### PEAR

PEAR is een paired-end read merger die reads samenvoegd tot een stuk DNA. PEAR heeft een extreem lage false positive rate volgens de [github pagina](https://github.com/tseemann/PEAR). Vanwege deze positieve aspecten wordt PEAR veel gebruikt met metagenomics illumina data.
PEAR kan worden geinstalleerd via [bioconda](https://anaconda.org/bioconda/pear). 
Maar het blijkt dat alle data die wij hebben gekregen minION data is en wij geen illumina data hebben, daarom kunnen wij PEAR niet gebruiken.

*10-3-2025*
### Nanoplot

Nanoplot is een kwality control tool voor nanopore data(bijv minION), het neemt een input bestand die wordt gegenereerd door minION tijdens het sequencing genaamd: sequencing_summary.txt. Doordat maar een bestand gebruikt voor het maken van figuren is de tool heel snel. Als output genereert Nanoplot meerdere bestanden die worden aangegeven bijv. fastq, sorted bam en fasta bestanden(https://github.com/wdecoster/NanoPlot)
Nanoplot zit in [bioconda](https://anaconda.org/bioconda/r-nanopore).
Verder is Nanoplot niet heel speciaal, het is handig om even te kijken naar hoe de data er uitziet maar het visualiseert niet meer daarnaast.

### minIONQC
minIONQC is vergelijkbaar met Nanoplot vanwege hetzelfde bestand nemen als input en vergelijkbare output bestanden genereren. De reden om minIONQC te gebruiken is omdat minIONQC ook flow cell analyse geeft als output.[Documentatie](https://github.com/roblanf/minion_qc).


### pycoQC
pycoQC gebruikt net als minIONQC en Nanoplot een sequencing_summary.txt om figuren te genereren. De figuren van pycoQC zijn interactief en zitten in een jupyter notebook. Verder geeft het dezelfde soort plots als Nanoplot. [Documentatie](https://a-slide.github.io/pycoQC/).

*11-3-2025*
### welke van de drie?
Als basis is het goed om te kiezen tussen pycoQC en Nanoplot, minIONQC moet alleen gebruikt worden voor extra flow cell analyse. Het beste lijkt mij om gewoon Nanoplot te gebruiken, omdat er al wat flow cell analyse lijkt te zijn en dat niet heel relevant is voor ons project en nonoplot direct werkt op de fastq files, wat mij makkelijker lijkt.

### Keuze
Het blijkt dat er al quality control is uitgevoerdt op alle data, maar wij willen nog wel de quality control controleren. De tool die was gebruikt voor de quality control was NanoQC, dus die gaan wij niet gebruiken. In te plaats daarvan gebruiken wij MinionQC, omdat wij de extra flow cell analyse willen zien.

*3-14-2025*

### Begin inlezen minimap

minimap2 is een readmapping tool die data aligned door overlaps te vinden tussen lange reads. Dit doet het efficient en zonder veel geheugen te gebruiken door het gebruik von k-mers (meer context in het logboek van [Tai](./tai_logbook.ipynb)). Vanwege deze kwaliteiten is er besloten om minimap2 te gebruiken.

Er is een referentie database nodig, hiervoor staat een bestand op assemblix: "/data/dataprocessing/MinIONData/all_bacteria.mmi". In dit bestand staan alle bekende bacterie genomen.
 
Ik gebruik het allign commando:

```bash
minimap2 -a ref.mmi reads.fq > alignment.sam
```

bronnen:
https://github.com/lh3/minimap2
https://lh3.github.io/minimap2/minimap2.html
 

*3-16-2025*

Doel: maak minimap2 rule.

Voor de rule neem ik code 99 van barcode01 als voorbeeld voor experimenteren met de rule. In de config noem ik deze "EXAMPLE_SAMPLE

Eind: Ik heb nu het skelet van een rule met de belangrijkste aspecten maar zonder functionaliteit.

*3-18-2025*
Na het maken van een minimap2 rule lijkt er een onverwachte error te zijn, omdat het de input file niet krijgt.
De error is nu opgelost het wat een fout in het expand commando met de extensies. 

Nu wil ik andere dingen toevoegen aan de rule zoals log en conda config. Ik moet ook nog het commando uitbreiden om alle files te gebruiken van alle barcodes.

*3-20-2025*
Na overleg met de anderen hebben wij besloten om kraken2 te gebruiken in te plaats van minimap2 vanwege kraken2's betere werking op 16S data. Om deze reden ga ik mijn minimap2 rule omzetten naar een Braken rule terwijl [Tai](./tai_logbook.ipynb) bezig gaat met de kraken2 rule.


## Bracken

voorbeeld commando volgens de officiele handleiding:

```{bash}
bracken -d {KRAKEN_DB} -i {SAMPLE}.kreport -o {SAMPLE}.bracken -r {READ_LEN} -l {CLASSIFICATION_LEVEL} -t {THRESHOLD}
```




[officiele handleiding](https://ccb.jhu.edu/software/bracken/index.shtml?t=manual)
[Documentatie](https://github.com/jenniferlu717/Bracken)

Bracken staat op [bioconda](https://anaconda.org/bioconda/bracken)

[Tai](tai_logbook.ipynb) heeft voorbeeld code voor kraken2 waar die ik kan gebruiken om bracken te testen.

Dus ik run:

```{bash}
cp /commons/Themas/Thema07/metagenomics/bioplastic/soil_metagenomics/Ariana/20240418_1521_MN35459_FAX77312_60832f1a/fastq_pass/barcode01/FAX77312_pass_barcode01_60832f1a_527e8671_0.fastq.gz ~/Documents/bracken_testdata
```
```{bash}
gunzip ~/Documents/bracken_testdata/FAX77312_pass_barcode01_60832f1a_527e8671_0.fastq.gz
```
```{bash}
head -n 800000 ~/Documents/bracken_testdata/FAX77312_pass_barcode01_60832f1a_527e8671_0.fastq > ~/Documents/bracken_testdata/test_data.fastq
```
```{bash}
kraken2 --db /data/datasets/KRAKEN2_INDEX/16S_Silva/ --threads 4 --report ~/Documents/bracken_testdata/subset_test.kreport2 --output ~/Documents/bracken_testdata/output.txt ~/Documents/bracken_testdata/test_data.fastq
```
ik heb nu een folder genaamd bracken_testdata waarin de kraken2 output files en de fastq files in staan.

Op de ouput die ik heb gegenereerd kan ik nu bracken uitproberen.

```{bash}
bracken -d /data/datasets/KRAKEN2_INDEX/16S_Silva/ -i ~/Documents/bracken_testdata/subset_test.kreport2 -o ~/Documents/bracken_out/subset_test.bracken 
```

Als ik bracken run op de output van kraken2 krijg ik de error :
Error: no reads found. Please check your Kraken report

als ik kijk naar de hele kreport2 file zie ik dat de file maar gaat tot het Class niveau, dit betekent dat als ik -l C ofzo toevoeg bracken wel het bestand kan aanpassen. Dus nu run ik:

```{bash}
bracken -d /data/datasets/KRAKEN2_INDEX/16S_Silva/ -i ~/Documents/bracken_testdata/subset_test.kreport2 -o ~/Documents/bracken_out/subset_test.bracken -l P
```

Ik krijg dezelfde error, nu ik beter kijk zijn 35 van de 36 read unclassified. 

Het ligt dus waarschijnlijk aan kraken2, dus ik kan andere databases proberen om te zien of die betere resultaten krijgen

Ik heb een gevonden:

```{bash}
kraken2 --db /data/datasets/KRAKEN2_INDEX/k2_standard/ --threads 4 --report ~/Documents/bracken_testdata/subset_test.kreport2 --output ~/Documents/bracken_testdata/output.txt ~/Documents/bracken_testdata/test_data.fastq
```

De k2_standard database classificeerd 33 van de 36 sequencies. Hier kan ik waarschijnlijk wel bracken op runnen.

```{bash}
bracken -d /data/datasets/KRAKEN2_INDEX/k2_standard/ -i ~/Documents/bracken_testdata/subset_test.kreport2 -o ~/Documents/bracken_out/subset_test.bracken 
```
Dit genereert het output bestand subset_test.bracken, dit bestand heeft een rij die van Streptomices. 
Bracken heeft ook het bestand subset_test_bracken_species.kreport2

Ik kan in de bracken input geven welk niveau wordt gebruikt met -l, dit is standaard S (Species), maar ik kan kan het ook hoger zetten.


De verschillende barcodes worden gefuseert tot een bestand, daar moet ik rekening mee houden in de bracken rule.

Ik moet nu ook nog een config maken voor de bracken rule

Nu de bracken rule af is wil ik verder gaan met phyloseq en een begin maken met de poster.

Phyloseq is een R-Package die visualisaties maakt, hiervoor gebruikt het ggplot. Ik heb een tutorial gevonden:
https://vaulot.github.io/tutorials/Phyloseq_tutorial.html
 
Om dit uit te proberen ga ik schrijven in een rmarkdown minilogboek.  

Er is een combined wgs kreport gemaakt, hier ga ik bracken op runnen:

```{bash}
bracken -d /data/datasets/KRAKEN2_INDEX/k2_standard/ -i ~/Documents/metagenomics/combined_wgs_barcode01_report.kreport2 -o /students/2024-2025/Thema07/metagenomics/bioplastic/combined_wgs_barcode01_report.bracken
```